In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna japanize-matplotlib > /dev/null

In [3]:
# パスを設定
my_path = "/content/drive/MyDrive/Colab Notebooks/Nishika 中古マンション価格予測2024春の部/"

In [4]:
cd {my_path}

/content/drive/MyDrive/Colab Notebooks/Nishika 中古マンション価格予測2024春の部


In [5]:
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns #データ可視化ライブラリ
sns.set(font="IPAexGothic")

from glob import glob #複数のファイルを扱えるライブラリ
from functools import partial
#　functools ある関数を操作したり、それによって新たな関数を取得することができる関数をまとめたモジュール
#  partial ある関数の引数やキーワードの一部を固定して新たな関数を作る
from itertools import product
#  itertools イテレータを構築する部品を実装したモジュール
#  product 直積(デカルト)をメソッド
#  直積(デカルト) 複数の集合から要素を一つづつ取り出した組み合わせの集合
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, train_test_split
#  KFold k分割交差検証
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, OrdinalEncoder


In [ ]:
!cat /proc/meminfo | awk '($1=="MemTotal:") {print $2/1048576}'

12.6748


In [ ]:
!python -V

Python 3.10.12


In [ ]:
!pip freeze
# pip Pythonパッケージ管理システム
# !pip freeze インストール済みのパッケージ一覧を確認

absl-py==1.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
alembic==1.13.1
altair==4.2.2
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.8.0
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.6.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.3
colorcet==3.1.0
colorlog==6.8.2
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.7
cuda-python==12.2.1
c

In [6]:
# データ処理
paths = glob(my_path + '/train/*')
tr_dfs = []
for path in paths:
  tr_df = pd.read_csv(path)
  tr_dfs.append(tr_df)
tr_df = pd.concat(tr_dfs)
tr_df.reset_index(drop=True, inplace=True)
te_df = pd.read_csv(my_path + '/test.csv')

<ipython-input-6-d1779be7a1c2>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tr_df = pd.read_csv(path)
<ipython-input-6-d1779be7a1c2>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tr_df = pd.read_csv(path)
<ipython-input-6-d1779be7a1c2>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tr_df = pd.read_csv(path)
<ipython-input-6-d1779be7a1c2>:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tr_df = pd.read_csv(path)


In [7]:
tr_df.head(5)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,28083619,中古マンション等,NaN,28101,兵庫県,神戸市東灘区,田中町,住吉(ＪＲ・六甲ライナー),8,３ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2015年第3四半期,未改装,NaN,7.380211
1,28193242,中古マンション等,NaN,28102,兵庫県,神戸市灘区,水道筋,王子公園,8,４ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,400.0,2008年第1四半期,改装済,NaN,7.505150
2,28083906,中古マンション等,NaN,28101,兵庫県,神戸市東灘区,深江南町,芦屋(阪神),7,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,NaN,NaN,2015年第1四半期,改装済,NaN,7.447158
3,28122186,中古マンション等,NaN,28206,兵庫県,芦屋市,大東町,打出,9,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2017年第2四半期,NaN,NaN,6.968483
4,28027066,中古マンション等,NaN,28202,兵庫県,尼崎市,今福,杭瀬,9,３ＤＫ,...,NaN,NaN,NaN,第２種住居地域,60.0,200.0,2019年第2四半期,NaN,NaN,6.653213


In [8]:
tr_df.isnull().sum()

ID                   0
種類                   0
地域              820017
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                661
最寄駅：名称            2837
最寄駅：距離（分）        23263
間取り              31804
面積（㎡）                0
土地の形状           820017
間口              820017
延床面積（㎡）         820017
建築年              24005
建物の構造            24408
用途               95741
今後の利用目的         369508
前面道路：方位         820017
前面道路：種類         820017
前面道路：幅員（ｍ）      820017
都市計画             21316
建ぺい率（％）          26181
容積率（％）           26181
取引時点                 0
改装               83074
取引の事情等          800350
取引価格（総額）_log         0
dtype: int64

In [9]:
sub_df = tr_df.copy()

In [ ]:
tr_df = sub_df.copy()
tr_df

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,28083619,中古マンション等,NaN,28101,兵庫県,神戸市東灘区,田中町,住吉(ＪＲ・六甲ライナー),8,３ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2015年第3四半期,未改装,NaN,7.380211
1,28193242,中古マンション等,NaN,28102,兵庫県,神戸市灘区,水道筋,王子公園,8,４ＬＤＫ,...,NaN,NaN,NaN,近隣商業地域,80.0,400.0,2008年第1四半期,改装済,NaN,7.505150
2,28083906,中古マンション等,NaN,28101,兵庫県,神戸市東灘区,深江南町,芦屋(阪神),7,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,NaN,NaN,2015年第1四半期,改装済,NaN,7.447158
3,28122186,中古マンション等,NaN,28206,兵庫県,芦屋市,大東町,打出,9,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2017年第2四半期,NaN,NaN,6.968483
4,28027066,中古マンション等,NaN,28202,兵庫県,尼崎市,今福,杭瀬,9,３ＤＫ,...,NaN,NaN,NaN,第２種住居地域,60.0,200.0,2019年第2四半期,NaN,NaN,6.653213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820012,25005276,中古マンション等,NaN,25201,滋賀県,大津市,柳が崎,大津京,16,NaN,...,NaN,NaN,NaN,商業地域,80.0,400.0,2022年第3四半期,NaN,NaN,7.623249
820013,25040242,中古マンション等,NaN,25201,滋賀県,大津市,竜が丘,膳所,6,４ＬＤＫ,...,NaN,NaN,NaN,準工業地域,60.0,200.0,2012年第3四半期,未改装,NaN,7.000000
820014,25021978,中古マンション等,NaN,25201,滋賀県,大津市,仰木の里,おごと温泉,19,４ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,200.0,2017年第2四半期,改装済,NaN,7.113943
820015,25030391,中古マンション等,NaN,25201,滋賀県,大津市,石山寺,石山,30分?60分,３ＬＤＫ,...,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2015年第4四半期,改装済,NaN,7.176091


In [10]:
# dropnaで欠損値を含む列を削除
tr_df = tr_df.dropna(axis = 'columns')

In [11]:
# 列ごとにデータ型確認
tr_df.dtypes

ID                int64
種類               object
市区町村コード           int64
都道府県名            object
市区町村名            object
面積（㎡）            object
取引時点             object
取引価格（総額）_log    float64
dtype: object

In [12]:
tr_df = tr_df.drop(['ID', '市区町村コード', '市区町村名'] , axis=1)
tr_df.head(5)

,種類,都道府県名,面積（㎡）,取引時点,取引価格（総額）_log
0,中古マンション等,兵庫県,65,2015年第3四半期,7.380211
1,中古マンション等,兵庫県,75,2008年第1四半期,7.505150
2,中古マンション等,兵庫県,75,2015年第1四半期,7.447158
3,中古マンション等,兵庫県,65,2017年第2四半期,6.968483
4,中古マンション等,兵庫県,50,2019年第2四半期,6.653213


In [13]:
print(tr_df['取引時点'].unique())

['2015年第3四半期' '2008年第1四半期' '2015年第1四半期' '2017年第2四半期' '2019年第2四半期'
 '2018年第1四半期' '2009年第1四半期' '2011年第1四半期' '2016年第2四半期' '2010年第4四半期'
 '2016年第3四半期' '2011年第4四半期' '2014年第4四半期' '2016年第1四半期' '2020年第1四半期'
 '2006年第3四半期' '2009年第3四半期' '2010年第1四半期' '2019年第4四半期' '2012年第3四半期'
 '2013年第4四半期' '2013年第1四半期' '2023年第1四半期' '2019年第1四半期' '2013年第2四半期'
 '2007年第1四半期' '2022年第3四半期' '2006年第4四半期' '2015年第2四半期' '2020年第4四半期'
 '2021年第1四半期' '2019年第3四半期' '2020年第2四半期' '2022年第4四半期' '2012年第2四半期'
 '2014年第1四半期' '2012年第4四半期' '2009年第2四半期' '2021年第2四半期' '2008年第3四半期'
 '2007年第2四半期' '2022年第2四半期' '2012年第1四半期' '2013年第3四半期' '2009年第4四半期'
 '2016年第4四半期' '2011年第3四半期' '2007年第4四半期' '2008年第2四半期' '2021年第3四半期'
 '2018年第2四半期' '2014年第2四半期' '2017年第1四半期' '2021年第4四半期' '2020年第3四半期'
 '2011年第2四半期' '2008年第4四半期' '2014年第3四半期' '2018年第3四半期' '2006年第2四半期'
 '2007年第3四半期' '2010年第3四半期' '2018年第4四半期' '2022年第1四半期' '2015年第4四半期'
 '2010年第2四半期' '2017年第4四半期' '2017年第3四半期' '2006年第1四半期' '2005年第4四半期'
 '2005年第3四半期']


In [14]:
time_list = [str(_) for _ in range(2005, 2024)]
time_list

['2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

In [15]:
for i in time_list:
  for j in range(len(tr_df)):
    if i in tr_df['取引時点'][j]:
      tr_df['取引時点'][j] = i


ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-15-145cb27354d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['取引時点'][j] = i
<ipython-input-15-145cb27354d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['取引時点'][j] = i
<ipython-input-15-145cb27354d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['取引時点'][j] = i
<ipython-input-15-145cb27354d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

In [16]:
tr_df.head(5)

,種類,都道府県名,面積（㎡）,取引時点,取引価格（総額）_log
0,中古マンション等,兵庫県,65,2015,7.380211
1,中古マンション等,兵庫県,75,2008,7.505150
2,中古マンション等,兵庫県,75,2015,7.447158
3,中古マンション等,兵庫県,65,2017,6.968483
4,中古マンション等,兵庫県,50,2019,6.653213


In [17]:
# 列ごとにデータ型確認
tr_df.dtypes

種類               object
都道府県名            object
面積（㎡）            object
取引時点             object
取引価格（総額）_log    float64
dtype: object

In [18]:
print(tr_df['面積（㎡）'].unique())

['65' '75' '50' '70' '95' '80' '40' '55' '60' '85' '45' '25' '90' '20'
 '105' '110' '15' '100' '145' '115' '370' '125' '140' '30' '165' '135'
 '35' '160' '120' '130' '250' '150' '10' '310' '760' '200' '190' '155'
 '195' '175' '330' '185' '210' '180' '290' '730' '240' '670' '2000㎡以上'
 '340' '170' '260' '320' '470' '390' '560' '220' '510' '450' '410' '300'
 55 85 25 60 65 70 90 80 120 75 95 20 50 30 45 360 15 145 110 40 1400 100
 900 130 35 125 105 135 115 10 190 165 490 175 140 160 520 155 150 185 170
 210 510 180 200 220 330 290 250 280 350 460 1000 230 270 860 320 240 260
 580 500 195 420 710 440 770 370 '700' '230' '740' '630' '400' '480' '500'
 '540' '440' '360' '600' '1300' '860' '680' '1500' '280' '270' '1100'
 '590' '1000' '420' '350' '1200' '720' '990' '770' 480 430 630 450 310 340
 380 560 1200 820 620 410 890 1100 730 700 400 300 390 690 760 680 970 870
 470 740 600 850 780 990 960 550 940 540 660 590 790 '820' '710' '980'
 '830' '620' '660' '650' '780' '490' '530' 570 650 950

In [21]:
for j in range(len(tr_df)):
    if tr_df['面積（㎡）'][j] == '2000㎡以上':
      tr_df['面積（㎡）'][j] = '2000'

<ipython-input-21-da000560e537>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['面積（㎡）'][j] = '2000'
<ipython-input-21-da000560e537>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['面積（㎡）'][j] = '2000'
<ipython-input-21-da000560e537>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tr_df['面積（㎡）'][j] = '2000'
<ipython-input-21-da000560e537>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [19]:
print(tr_df['面積（㎡）'].unique())

['65' '75' '50' '70' '95' '80' '40' '55' '60' '85' '45' '25' '90' '20'
 '105' '110' '15' '100' '145' '115' '370' '125' '140' '30' '165' '135'
 '35' '160' '120' '130' '250' '150' '10' '310' '760' '200' '190' '155'
 '195' '175' '330' '185' '210' '180' '290' '730' '240' '670' '2000㎡以上'
 '340' '170' '260' '320' '470' '390' '560' '220' '510' '450' '410' '300'
 55 85 25 60 65 70 90 80 120 75 95 20 50 30 45 360 15 145 110 40 1400 100
 900 130 35 125 105 135 115 10 190 165 490 175 140 160 520 155 150 185 170
 210 510 180 200 220 330 290 250 280 350 460 1000 230 270 860 320 240 260
 580 500 195 420 710 440 770 370 '700' '230' '740' '630' '400' '480' '500'
 '540' '440' '360' '600' '1300' '860' '680' '1500' '280' '270' '1100'
 '590' '1000' '420' '350' '1200' '720' '990' '770' 480 430 630 450 310 340
 380 560 1200 820 620 410 890 1100 730 700 400 300 390 690 760 680 970 870
 470 740 600 850 780 990 960 550 940 540 660 590 790 '820' '710' '980'
 '830' '620' '660' '650' '780' '490' '530' 570 650 950

In [ ]:
origin_df = tr_df.copy()

In [ ]:
origin_df.head(3)

,種類,都道府県名,面積（㎡）,取引時点,取引価格（総額）_log
0,中古マンション等,兵庫県,65,2015,7.380211
1,中古マンション等,兵庫県,75,2008,7.505150
2,中古マンション等,兵庫県,75,2015,7.447158


In [22]:
tr_df["取引時点"] = tr_df["取引時点"].astype(int)
tr_df["面積（㎡）"] = tr_df["面積（㎡）"].astype(int)


tr_df.dtypes

種類               object
都道府県名            object
面積（㎡）             int64
取引時点              int64
取引価格（総額）_log    float64
dtype: object

In [23]:
# 文字列をダミー変数へ変換
tr_df = pd.get_dummies(tr_df, drop_first = True, dtype=int)
tr_df.head(5)

,面積（㎡）,取引時点,取引価格（総額）_log,都道府県名_京都府,都道府県名_佐賀県,都道府県名_兵庫県,都道府県名_北海道,都道府県名_千葉県,都道府県名_和歌山県,都道府県名_埼玉県,...,都道府県名_群馬県,都道府県名_茨城県,都道府県名_長崎県,都道府県名_長野県,都道府県名_青森県,都道府県名_静岡県,都道府県名_香川県,都道府県名_高知県,都道府県名_鳥取県,都道府県名_鹿児島県
0,65,2015,7.380211,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,75,2008,7.505150,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,75,2015,7.447158,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,65,2017,6.968483,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50,2019,6.653213,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
# 列ごとにデータ型確認
tr_df.dtypes

面積（㎡）             int64
取引時点              int64
取引価格（総額）_log    float64
都道府県名_京都府         int64
都道府県名_佐賀県         int64
都道府県名_兵庫県         int64
都道府県名_北海道         int64
都道府県名_千葉県         int64
都道府県名_和歌山県        int64
都道府県名_埼玉県         int64
都道府県名_大分県         int64
都道府県名_大阪府         int64
都道府県名_奈良県         int64
都道府県名_宮城県         int64
都道府県名_宮崎県         int64
都道府県名_富山県         int64
都道府県名_山口県         int64
都道府県名_山形県         int64
都道府県名_山梨県         int64
都道府県名_岐阜県         int64
都道府県名_岡山県         int64
都道府県名_岩手県         int64
都道府県名_島根県         int64
都道府県名_広島県         int64
都道府県名_徳島県         int64
都道府県名_愛媛県         int64
都道府県名_愛知県         int64
都道府県名_新潟県         int64
都道府県名_東京都         int64
都道府県名_栃木県         int64
都道府県名_沖縄県         int64
都道府県名_滋賀県         int64
都道府県名_熊本県         int64
都道府県名_石川県         int64
都道府県名_神奈川県        int64
都道府県名_福井県         int64
都道府県名_福岡県         int64
都道府県名_福島県         int64
都道府県名_秋田県         int64
都道府県名_群馬県         int64
都道府県名_茨城県         int64
都道府県名_長崎県       

In [29]:
# データを学習、検証、テスト用に分割　6:2:2
train, test_valid = train_test_split(tr_df, test_size=0.4, random_state=1)
test, valid = train_test_split(test_valid, test_size=0.5, random_state=1)

In [30]:
# 学習データ
tr_x = train.drop(['取引価格（総額）_log'], axis=1)
tr_y = train['取引価格（総額）_log']

# 検証データ
val_x = valid.drop(['取引価格（総額）_log'], axis=1)
val_y = valid['取引価格（総額）_log']

# テストデータ
te_x = test.drop(['取引価格（総額）_log'], axis=1)
te_y = test['取引価格（総額）_log']

In [31]:
for i in tr_df.columns:
  print(i)

面積（㎡）
取引時点
取引価格（総額）_log
都道府県名_京都府
都道府県名_佐賀県
都道府県名_兵庫県
都道府県名_北海道
都道府県名_千葉県
都道府県名_和歌山県
都道府県名_埼玉県
都道府県名_大分県
都道府県名_大阪府
都道府県名_奈良県
都道府県名_宮城県
都道府県名_宮崎県
都道府県名_富山県
都道府県名_山口県
都道府県名_山形県
都道府県名_山梨県
都道府県名_岐阜県
都道府県名_岡山県
都道府県名_岩手県
都道府県名_島根県
都道府県名_広島県
都道府県名_徳島県
都道府県名_愛媛県
都道府県名_愛知県
都道府県名_新潟県
都道府県名_東京都
都道府県名_栃木県
都道府県名_沖縄県
都道府県名_滋賀県
都道府県名_熊本県
都道府県名_石川県
都道府県名_神奈川県
都道府県名_福井県
都道府県名_福岡県
都道府県名_福島県
都道府県名_秋田県
都道府県名_群馬県
都道府県名_茨城県
都道府県名_長崎県
都道府県名_長野県
都道府県名_青森県
都道府県名_静岡県
都道府県名_香川県
都道府県名_高知県
都道府県名_鳥取県
都道府県名_鹿児島県


In [32]:
# パラメータ設定
params = {
    'objective':'regression',
    'metric':'mae',
    'n_estimators':10000,
    'verbosity': -1,
    'random_state':42
}

# インスタンスの作成 (
model = lgb.LGBMRegressor(**params)  # 回帰であればlgb.LGBMRegressor()

# モデルの学習
model.fit(tr_x, tr_y, eval_metric='mae', eval_set=[(val_x, val_y)],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=True)]
       )

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[450]	valid_0's l1: 0.185457


LGBMRegressor(metric='mae', n_estimators=10000, objective='regression',
              random_state=42, verbosity=-1)

In [33]:
predict = model.predict(te_x)

In [34]:
from sklearn.metrics import mean_absolute_error


mae = mean_absolute_error(te_y, predict)
print('MAE : {:.3f}'.format(mae))

MAE : 0.186


In [35]:
te_df.head(3)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等
0,1000107,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,2,３ＬＤＫ,...,住宅,NaN,NaN,NaN,近隣商業地域,80.0,300.0,2023年第3四半期,改装済,NaN
1,1000109,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,0,１ＬＤＫ,...,その他,NaN,NaN,NaN,商業地域,80.0,400.0,2023年第3四半期,未改装,NaN
2,1000110,中古マンション等,NaN,1101,北海道,札幌市中央区,大通西,西１８丁目,0,NaN,...,NaN,NaN,NaN,NaN,商業地域,80.0,400.0,2023年第3四半期,NaN,NaN


In [36]:
te_df.isnull().sum()

ID                0
種類                0
地域            18738
市区町村コード           0
都道府県名             0
市区町村名             0
地区名               0
最寄駅：名称           21
最寄駅：距離（分）        21
間取り            1228
面積（㎡）             0
土地の形状         18738
間口            18738
延床面積（㎡）       18738
建築年             570
建物の構造           931
用途             4278
今後の利用目的         824
前面道路：方位       18738
前面道路：種類       18738
前面道路：幅員（ｍ）    18738
都市計画            265
建ぺい率（％）         358
容積率（％）          358
取引時点              0
改装             3100
取引の事情等        18597
dtype: int64

In [37]:
# dropnaで欠損値を含む列を削除
te_df = te_df.dropna(axis = 'columns')
te_df.head(3)

,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,面積（㎡）,取引時点
0,1000107,中古マンション等,1101,北海道,札幌市中央区,大通西,70,2023年第3四半期
1,1000109,中古マンション等,1101,北海道,札幌市中央区,大通西,35,2023年第3四半期
2,1000110,中古マンション等,1101,北海道,札幌市中央区,大通西,20,2023年第3四半期


In [38]:
check_x = te_df.drop(['ID', '地区名', "市区町村名", "市区町村コード", "種類"],axis=1)
check_x

,都道府県名,面積（㎡）,取引時点
0,北海道,70,2023年第3四半期
1,北海道,35,2023年第3四半期
2,北海道,20,2023年第3四半期
3,北海道,35,2023年第2四半期
4,北海道,20,2023年第2四半期
...,...,...,...
18733,沖縄県,75,2023年第2四半期
18734,沖縄県,45,2023年第3四半期
18735,沖縄県,85,2023年第2四半期
18736,沖縄県,60,2023年第2四半期


In [39]:
for i in time_list:
  for j in range(len(check_x)):
    if i in check_x['取引時点'][j]:
      check_x['取引時点'][j] = i

In [40]:
check_x

,都道府県名,面積（㎡）,取引時点
0,北海道,70,2023
1,北海道,35,2023
2,北海道,20,2023
3,北海道,35,2023
4,北海道,20,2023
...,...,...,...
18733,沖縄県,75,2023
18734,沖縄県,45,2023
18735,沖縄県,85,2023
18736,沖縄県,60,2023


In [41]:
print(check_x['面積（㎡）'].unique())

['70' '35' '20' '30' '75' '40' '15' '65' '50' '85' '80' '60' '55' '90'
 '45' '95' '105' '25' '100' '115' '120' '110' '140' '920' '135' '155'
 '125' '320' '170' '130' '160' '150' '210' '180' '10' '165' '200' '195'
 '145' '220' '190' '2000㎡以上' '1400' '710' '560']


In [42]:
for j in range(len(check_x)):
    if check_x['面積（㎡）'][j] == '2000㎡以上':
      check_x['面積（㎡）'][j] = '2000'

In [43]:
print(check_x['面積（㎡）'].unique())

['70' '35' '20' '30' '75' '40' '15' '65' '50' '85' '80' '60' '55' '90'
 '45' '95' '105' '25' '100' '115' '120' '110' '140' '920' '135' '155'
 '125' '320' '170' '130' '160' '150' '210' '180' '10' '165' '200' '195'
 '145' '220' '190' '2000' '1400' '710' '560']


In [44]:
# 文字列を数値データに置換
check_x["取引時点"] = check_x["取引時点"].astype(int)
check_x["面積（㎡）"] = check_x["面積（㎡）"].astype(int)


check_x.dtypes

都道府県名    object
面積（㎡）     int64
取引時点      int64
dtype: object

In [45]:
# 文字列をダミー変数へ変換
check_x = pd.get_dummies(check_x, drop_first = True, dtype=int)
check_x.head(5)

,面積（㎡）,取引時点,都道府県名_京都府,都道府県名_佐賀県,都道府県名_兵庫県,都道府県名_北海道,都道府県名_千葉県,都道府県名_和歌山県,都道府県名_埼玉県,都道府県名_大分県,...,都道府県名_群馬県,都道府県名_茨城県,都道府県名_長崎県,都道府県名_長野県,都道府県名_青森県,都道府県名_静岡県,都道府県名_香川県,都道府県名_高知県,都道府県名_鳥取県,都道府県名_鹿児島県
0,70,2023,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35,2023,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20,2023,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,35,2023,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,20,2023,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
check_x.dtypes

面積（㎡）         int64
取引時点          int64
都道府県名_京都府     int64
都道府県名_佐賀県     int64
都道府県名_兵庫県     int64
都道府県名_北海道     int64
都道府県名_千葉県     int64
都道府県名_和歌山県    int64
都道府県名_埼玉県     int64
都道府県名_大分県     int64
都道府県名_大阪府     int64
都道府県名_奈良県     int64
都道府県名_宮城県     int64
都道府県名_宮崎県     int64
都道府県名_富山県     int64
都道府県名_山口県     int64
都道府県名_山形県     int64
都道府県名_山梨県     int64
都道府県名_岐阜県     int64
都道府県名_岡山県     int64
都道府県名_岩手県     int64
都道府県名_島根県     int64
都道府県名_広島県     int64
都道府県名_徳島県     int64
都道府県名_愛媛県     int64
都道府県名_愛知県     int64
都道府県名_新潟県     int64
都道府県名_東京都     int64
都道府県名_栃木県     int64
都道府県名_沖縄県     int64
都道府県名_滋賀県     int64
都道府県名_熊本県     int64
都道府県名_石川県     int64
都道府県名_神奈川県    int64
都道府県名_福井県     int64
都道府県名_福岡県     int64
都道府県名_福島県     int64
都道府県名_秋田県     int64
都道府県名_群馬県     int64
都道府県名_茨城県     int64
都道府県名_長崎県     int64
都道府県名_長野県     int64
都道府県名_青森県     int64
都道府県名_静岡県     int64
都道府県名_香川県     int64
都道府県名_高知県     int64
都道府県名_鳥取県     int64
都道府県名_鹿児島県    int64
dtype: object

In [47]:
check_y = model.predict(check_x)

In [48]:
check_y

array([7.23180658, 6.69739467, 6.47084151, ..., 7.55930522, 7.32247777,
       7.41567279])

In [49]:
check_y[0]

7.23180658031967

In [50]:
# ファイルへの上書き
df_check = pd.read_csv(my_path + '/sample_submission.csv')

for i in range(len(check_y)):
  df_check['取引価格（総額）_log'][i] = check_y[i]

ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-50-16b0d68b887c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check['取引価格（総額）_log'][i] = check_y[i]
<ipython-input-50-16b0d68b887c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check['取引価格（総額）_log'][i] = check_y[i]
<ipython-input-50-16b0d68b887c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check['取引価格（総額）_log'][i] = check_y[i]
<ipython-input-50-16b0d68b887c>:5: Settin

In [51]:
df_check

,ID,取引価格（総額）_log
0,1000107,7.231807
1,1000109,6.697395
2,1000110,6.470842
3,1000114,6.697395
4,1000115,6.470842
...,...,...
18733,47003396,7.520882
18734,47005356,6.984957
18735,47005578,7.559305
18736,47005892,7.322478


In [52]:
df_check.to_csv(my_path + '/sample_submission.csv', index=False)